# Semantické jadro s integráciou OpenBnB MCP servera

Tento notebook demonštruje, ako používať Semantické jadro s aktuálnym OpenBnB MCP serverom na vyhľadávanie skutočných ubytovaní Airbnb pomocou MCPStdioPlugin. Na prístup k LLM využíva Azure AI Foundry. Na nastavenie vašich environmentálnych premenných môžete postupovať podľa [lekcie nastavenia](/00-course-setup/README.md).


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Vytvorenie pripojenia MCP Plugin

Pripojíme sa k [OpenBnB MCP serveru](https://github.com/openbnb-org/mcp-server-airbnb) pomocou MCPStdioPlugin. Tento server poskytuje funkciu vyhľadávania Airbnb prostredníctvom balíka @openbnb/mcp-server-airbnb.


## Vytvorenie klienta

V tomto príklade použijeme Azure AI Foundry na prístup k LLM. Uistite sa, že vaše environmentálne premenné sú správne nastavené.


## Konfigurácia prostredia

Nastavte parametre Azure OpenAI. Uistite sa, že máte nastavené nasledujúce premenné prostredia:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Pochopenie integrácie OpenBnB MCP

Tento notebook sa pripája k **reálnemu serveru OpenBnB MCP**, ktorý poskytuje skutočnú funkčnosť vyhľadávania na Airbnb.

### Ako to funguje:

1. **MCPStdioPlugin**: Používa komunikáciu cez štandardný vstup/výstup so serverom MCP
2. **Reálny NPM balík**: Stiahne a spustí `@openbnb/mcp-server-airbnb` pomocou npx
3. **Živé dáta**: Vracia skutočné údaje o nehnuteľnostiach z Airbnb API
4. **Objavovanie funkcií**: Agent automaticky objavuje dostupné funkcie zo servera MCP

### Dostupné funkcie:

Server OpenBnB MCP zvyčajne poskytuje:
- **search_listings** - Vyhľadávanie nehnuteľností na Airbnb podľa lokality a kritérií
- **get_listing_details** - Získanie podrobných informácií o konkrétnych nehnuteľnostiach
- **check_availability** - Kontrola dostupnosti na konkrétne dátumy
- **get_reviews** - Získanie recenzií na nehnuteľnosti
- **get_host_info** - Získanie informácií o hostiteľoch nehnuteľností

### Predpoklady:

- **Node.js** nainštalovaný vo vašom systéme
- **Internetové pripojenie** na stiahnutie balíka MCP servera
- **NPX** dostupné (je súčasťou Node.js)

### Testovanie pripojenia:

Pripojenie k serveru MCP môžete otestovať manuálne spustením:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Týmto sa stiahne a spustí server OpenBnB MCP, ku ktorému sa Semantic Kernel následne pripojí pre získanie reálnych údajov z Airbnb.


## Spustenie agenta s OpenBnB MCP serverom

Teraz spustíme AI agenta, ktorý sa pripája k OpenBnB MCP serveru, aby vyhľadal skutočné ubytovania Airbnb v Štokholme pre 2 dospelých a 1 dieťa. Neváhajte upraviť zoznam `user_inputs`, aby ste zmenili kritériá vyhľadávania.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Zhrnutie  
Gratulujeme! Úspešne ste vytvorili AI agenta, ktorý integruje vyhľadávanie ubytovania v reálnom svete pomocou Model Context Protocol (MCP):

Použité technológie:  
- Semantic Kernel – Na vytváranie inteligentných agentov s Azure OpenAI  
- Azure AI Foundry – Pre schopnosti LLM a dokončovanie konverzácií  
- MCP (Model Context Protocol) – Na štandardizovanú integráciu nástrojov  
- OpenBnB MCP Server – Pre skutočnú funkčnosť vyhľadávania na Airbnb  
- Node.js/NPX – Na spustenie externého MCP servera  

Čo ste sa naučili:  
- Integrácia MCP: Prepojenie agentov Semantic Kernel s externými MCP servermi  
- Prístup k dátam v reálnom čase: Vyhľadávanie skutočných nehnuteľností na Airbnb cez živé API  
- Komunikácia cez protokol: Používanie stdio komunikácie medzi agentom a MCP serverom  
- Objavovanie funkcií: Automatické zisťovanie dostupných funkcií z MCP serverov  
- Streamovanie odpovedí: Zachytávanie a zaznamenávanie volaní funkcií v reálnom čase  
- Renderovanie HTML: Formátovanie odpovedí agenta pomocou štýlovaných tabuliek a interaktívnych zobrazení  

Ďalšie kroky:  
- Integrovať ďalšie MCP servery (počasie, lety, reštaurácie)  
- Vytvoriť multi-agentový systém kombinujúci MCP a A2A protokoly  
- Vytvoriť vlastné MCP servery pre vaše vlastné zdroje dát  
- Implementovať perzistentnú pamäť konverzácií medzi reláciami  
- Nasadiť agenta do Azure Functions s orchestráciou MCP serverov  
- Pridať autentifikáciu používateľov a možnosti rezervácie  

Kľúčové výhody MCP architektúry:  
- Štandardizácia: Univerzálny protokol na prepojenie AI agentov s externými nástrojmi  
- Dáta v reálnom čase: Prístup k aktuálnym, najnovším informáciám z rôznych služieb  
- Rozšíriteľnosť: Jednoduchá integrácia nových zdrojov dát a nástrojov  
- Interoperabilita: Funguje naprieč rôznymi AI rámcami a platformami agentov  
- Oddelenie zodpovedností: Jasné rozlíšenie medzi AI logikou a prístupom k externým dátam  



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
